# Trabalho 3: Avaliando sistemas de IR

Neste trabalho, o aluno irá explorar o problema de otimização de sistemas de IR através da análise de métricas de avaliação populares para esse tipo de sistema.

Para isso, vamos usar a base CFC (com artigos sobre fibrose cística), a qual encontra-se nesta pasta.

Observação: a idéia aqui é usar o jupyter para mostrar a evolução dos teus experimentos. Então use ele adequadamente. Ou seja, não altere funções do começo para reexecutar algo que foi feito lá embaixo. Neste caso, sobreescreva a função lá embaixo. A leitura do notebook tem que ser feita sequencialmente para ficar fácil, ok?


## Passo 1 - Normalização

Primeira coisa que você vai precisar fazer é ler os arquivos do CFC. Essa é uma parte meio braçal mesmo, mas não tem como escapar dela. Então você terá que criar funções que consigam ler cada arquivo do CFC e parsear eles.


In [1]:
import pandas as pd

# Imprime os dados no formato JSON
def printJson(data):
    print(json.dumps(data, indent=4))
    
def takeNumber(data):
    return data['number']

def takeSecond(data):
    return data[1]

# Extrai artigos de um arquivo CFC
def extractCFCArticles(path):
    # Lista de campos do arquivo CFC
    fields = [
            'PN',
            'RN',
            'AN',
            'AU',
            'TI',
            'SO',
            'MJ',
            'MN',
            'AB',
            'EX',
            'RF',
            'CT',
    ]
    
    # Lista de campos que mudarao seus nomes
    cast = {
        'RN': 'record',
        'AB': 'content',
        'EX': 'content',
        'AN': 'an',
        'TI': 'title'
    }
    file = open(path).read()
    
    #Lista de artigos extraidos
    articles = []

    for article in file.split("\n\n"):
        # Objeto do artigo
        articleData = {
            'title':'',
            'record': 0,
            'year': 0,
            'number': 0,
            'an': 0,
            'AU': '',
            'SO': '',
            'MJ': [],
            'MN': [],
            'RF': [],
            'CT': [],
            'contentType': '',
            'content': '',
        }
        
        currentField = ''
        
        for line in article.split("\n"):
            # Linha vazia ou fim de arquivo
            if(not line.strip() or line.startswith("\x1a")):
                continue 
            
            for key in fields:
                # Verificase a linha é a definição de um novo atributo
                if(line.startswith(key + ' ')):
                    currentField = key
                    line = line[3:]
            
            line = line.strip()
            
            if(currentField == 'RF' or currentField == 'CT'):
                articleData[currentField].append(line)
                continue
                
            if(currentField == 'MJ' or currentField == 'MN'):
                articleData[currentField]+= line.split("  ")
                continue
            
            if(currentField == 'PN'):
                articleData['year'] = int(line[0:2])
                articleData['number'] = int(line[2])
                continue
            
            if(currentField == 'AB' or currentField == 'EX'):
                articleData['contentType'] = currentField
                
            if(currentField in cast):
                currentField = cast[currentField]
            
            if(currentField == 'record' or currentField == 'an'):
                articleData[currentField] = int(line)
                continue
                
                
            if(articleData[currentField]):
                articleData[currentField]+=' '
            articleData[currentField]+= line 
        
        if(articleData['title']):
            articles.append(articleData)
    return articles
            
def extractQueryFile(file):
    fields = [
        "QN",
        "QU",
        "NR",
        "RD"
    ]
    
    queriesList = open(file).read().split("\n")
    for i, line in enumerate(queriesList):
        queriesList[i] = line.strip()
    queriesList = "\n".join(queriesList).split("\n\n")
    
    queries = []
    for query in queriesList:
        query = query.strip()
        if not query: continue
        
        queryData = {
            'number':0,
            'text': '',
            'length': 0,
            'docs': [],
        }
        
        lines = query.split("\n")
        
        currentField = ''
        
        for line in query.split("\n"):
            for key in fields:
                # Verificase a linha é a definição de um novo atributo
                if(line.startswith(key + ' ')):
                    currentField = key
                    line = line[3:]
                line = line.strip()

            if currentField == fields[0] or currentField == fields[2]:
                queryData['number' if currentField == fields[0] else 'length'] = int(line)
            elif currentField == fields[1]:
                if(queryData['text']):
                    queryData['text']+=' '
                queryData['text']+= line
            elif currentField == fields[3]:
                values = line.split(' ')
                doc = []
                for value in values:
                    if not value: continue
                    doc.append(value)
                    if len(doc) == 2:
                        doc[0] = int(doc[0])
                        weights = doc[1]
                        doc[1] = (int(weights[0]) + int(weights[1]) + int(weights[2]) + int(weights[3]))/4
                        queryData['docs'].append(doc)
                        doc = []
        queryData['docs'].sort(key=takeSecond,reverse=True)
        queries.append(queryData)
    queries.sort(key=takeNumber)
    return queries
                    
    
articles = [
    'data/cf74',
    'data/cf75',
    'data/cf76',
    'data/cf77',
    'data/cf78',
    'data/cf79'
]

queryFile = 'data/cfquery'

import json
data = extractCFCArticles(articles[5])

# printJson(extractQueryFile(queryFile))

pd.DataFrame(extractQueryFile(queryFile))
# for i in data:
#     print(i['AB'] and i['EX'])
#     print(i, "\n--------------------------------------------------\n")

,docs,length,number,text
0,"[[533, 2.0], [139, 1.75], [441, 1.75], [151, 1...",34,1,What are the effects of calcium on the physica...
1,"[[875, 0.75], [434, 0.5], [592, 0.5], [169, 0....",7,2,Can one distinguish between the effects of muc...
2,"[[633, 2.0], [139, 1.75], [375, 1.75], [856, 1...",43,3,How are salivary glycoproteins from CF patient...
3,"[[604, 2.0], [876, 2.0], [711, 1.75], [669, 0....",9,4,What is the lipid composition of CF respirator...
4,"[[151, 2.0], [374, 2.0], [439, 2.0], [593, 2.0...",131,5,Is CF mucus abnormal?
5,"[[31, 2.0], [503, 1.25], [875, 1.0], [370, 0.5...",24,6,What is the effect of water or other therapeut...
6,"[[371, 1.75], [623, 1.5], [856, 1.25], [1064, ...",28,7,Are mucus glycoproteins degraded differently i...
7,"[[23, 1.5], [499, 1.5], [683, 1.5], [1125, 1.5...",22,8,What histochemical differences have been descr...
8,"[[414, 2.0], [165, 1.5], [794, 1.0], [1115, 0....",10,9,What is the association between liver disease ...
9,"[[157, 2.0], [413, 2.0], [581, 2.0], [676, 1.7...",25,10,What is the role of Vitamin E in the therapy o...


## Passo 2 - Primeira indexação

Depois de parsear os arquivos, agora é hora de indexar os dados. Ao invés de criar o código do zero, vamos usar o Whoosh, que é uma implementação em python inspirada no Lucene. A documentação pode ser encontrada em https://whoosh.readthedocs.io/en/latest/index.html.

Nesta questão de indexação, temos que fazer algumas escolhas: 
* Como fazer o processo de tokenização? 
* Stemmizar ou não stemmizar, eis a questão...
* Quais campos são úteis para indexação?

Neste momento, você vai tomar suas decisões iniciais. A ideia é você testar, depois, outras soluções para verificar quais tiveram os melhores resultados, entendeu? Então não esqueça de documentar, aqui, qual a sua decisão inicial e depois ir explicando ao longo do notebook os experimentos que está fazendo.

Para pesquisa e indexação, iremos avaliar principalmente os subjects. Estes, levantados por especialistas, contém informações sobre palavras-chave que definem os artigos. 

No geral, cada artigo possui um Major e Minor Subject. O primeiro define o foco do artigo enquanto o segundo, temas secundários abordados.

Dentro dos subjects, os termos são definidos seguindo o [vocabulário médico MeSH](https://meshb.nlm.nih.gov/), em alguns casos seguido de uma sigla de qualificadores (Ex di:diagnóstico).

A decisão inicial é usar destas informações para gerar indexação e ranqueamento de pesquisas. Priorizando o Major subject junto do qualificador, em seguida minor subjects

Usaremos também os autores e citações para calcular o H-index dos artigos

In [2]:
import os
from os import mkdir
from shutil import rmtree
from whoosh.index import create_in
from whoosh.fields import *

db_name = "indexdir"

schema = Schema(title=TEXT(stored=True), 
                record=NUMERIC(stored=True), 
                year=NUMERIC(stored=True),
                number=NUMERIC(stored=True),
                an=NUMERIC(stored=True),
                content=TEXT,
                mn=TEXT,
                mj=TEXT,
               )

if os.path.exists(db_name):
    rmtree(db_name)
os.mkdir(db_name)
ix = create_in(db_name, schema)
writer = ix.writer()
articlesLength = 0

for file in articles:
    for article in extractCFCArticles(file):
        articlesLength+=1
        mn = ""
        mj = ""
        for keyWord in article['MN']:
            mn+= ' ' + keyWord
            
        for keyWord in article['MJ']:
            mj+= ' ' + keyWord
        
        mn = mn.strip()
        mj = mj.strip()

        writer.add_document(title=article['title'],
                            record=article['record'],
                            year=article['year'],
                            number=article['number'],
                            an=article['an'],
                            content=article['content'],
                            mn=mn,
                            mj=mj
                           )

writer.commit()

print("OK")
print(articlesLength)


OK
1239


In [4]:
from whoosh.qparser import QueryParser
from whoosh.qparser import MultifieldParser
import whoosh

searcher = None

# Pesquisa inicial
def search(query):
    global searcher
    searcher = ix.searcher()
    og = whoosh.qparser.OrGroup.factory(0.9)
    parser = MultifieldParser(["title", "content", "mn", "mj"], schema=ix.schema, group=og)
    query = parser.parse("What are the effects of calcium on the physical properties of mucus from CF patients?")
    results = searcher.search(query, limit=None)
    return results

def searchClose():
    global searcher
    searcher.close()
    

prints = []
for query in extractQueryFile(queryFile):
    data = search(query['text'])
    results = data
    printData = {
        'QueryNumber': query['number'],
        'totalEsperado': query['length'],
        'totalRetornado': len(results),
        'docs': [],
    }
    
    for i, result in enumerate(query['docs']):
        printData['docs'].append([
            query['docs'][i][0],
            query['docs'][i][1],
            results[i]['record']
        ])
    prints.append(printData)
    searchClose()

# printJson(prints[42])
pd.DataFrame(prints)


,QueryNumber,docs,totalEsperado,totalRetornado
0,1,"[[533, 2.0, 533], [139, 1.75, 975], [441, 1.75...",34,541
1,2,"[[875, 0.75, 533], [434, 0.5, 975], [592, 0.5,...",7,541
2,3,"[[633, 2.0, 533], [139, 1.75, 975], [375, 1.75...",43,541
3,4,"[[604, 2.0, 533], [876, 2.0, 975], [711, 1.75,...",9,541
4,5,"[[151, 2.0, 533], [374, 2.0, 975], [439, 2.0, ...",131,541
5,6,"[[31, 2.0, 533], [503, 1.25, 975], [875, 1.0, ...",24,541
6,7,"[[371, 1.75, 533], [623, 1.5, 975], [856, 1.25...",28,541
7,8,"[[23, 1.5, 533], [499, 1.5, 975], [683, 1.5, 7...",22,541
8,9,"[[414, 2.0, 533], [165, 1.5, 975], [794, 1.0, ...",10,541
9,10,"[[157, 2.0, 533], [413, 2.0, 975], [581, 2.0, ...",25,541


## Passo 3 - Calculando as métricas de avaliação

Para avaliar a tua decisão, vamos usar como métricas a precisão, recall e f-measure. Faça uma variação da precisão e recall como P@n e R@n, pois serão úteis para os gráficos que serão gerados. 

In [13]:
#precisão e recall usando a média 
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

def extractSearchIds(results):
    ids = []
    for doc in results:
        ids.append(doc['record'])
    return ids

def extractQueryIds(query):
    ids = []
    for doc in query['docs']:
        ids.append(doc[0])
    return ids

def getTruePositives(searchIds, queryIds):
    return len(set(searchIds) & set(queryIds))

def getFalsePositives(searchIds, queryIds):
    return len(searchIds) - getTruePositives(searchIds, queryIds)

def getFalseNegatives(searchIds, queryIds):
    return len(queryIds) - getTruePositives(searchIds, queryIds)

def getTrueNegatives(searchIds, queryIds):
    return articlesLength + getTruePositives(searchIds, queryIds) - getFalseNegatives(searchIds, queryIds) - getFalsePositives(searchIds, queryIds)

def getPrecision(searchIds, queryIds):
    return getTruePositives(searchIds, queryIds) / (getTruePositives(searchIds, queryIds) + 
                                                    getFalsePositives(searchIds, queryIds))

def getRecall(searchIds, queryIds):
    return getTruePositives(searchIds, queryIds) / (getTruePositives(searchIds, queryIds) +
                                                    getFalseNegatives(searchIds, queryIds))


def getFMeasures(searchIds, queryIds):
    return (2 * ( getRecall(searchIds, queryIds) * getPrecision(searchIds, queryIds) )) / (getPrecision(searchIds, queryIds) + 
                                                                                           getRecall(searchIds, queryIds) )


prints = []
for query in extractQueryFile(queryFile):
    results = search(query['text'])
    
    searchIds = extractSearchIds(results)
    queryIds = extractQueryIds(query)
    
    
    printData = {
        'len': query['length'],
        'truePositive': getTruePositives(searchIds, queryIds),
        'trueNegative': getTrueNegatives(searchIds, queryIds),
        'falsePositive':getFalsePositives(searchIds, queryIds),
        'falseNegative':getFalseNegatives(searchIds, queryIds),
        'precision':getPrecision(searchIds, queryIds),
        'recall':getRecall(searchIds, queryIds),
        'f-measure':getFMeasures(searchIds, queryIds),
    }
    
    prints.append(printData)
    searchClose()

print("Results: ")
# printJson(prints)
# pd.DataFrame(prints)
print("pronto")

Results: 
pronto


## Passo 4 - Gráficos do experimento 1

Apresentar agora os gráficos do teu experimento. Você deve gerar um gráfico PxR, como visto em sala. Para gerar gráficos, você pode usar o matplotlib. Se quiser usar outra biblioteca que ache mais fácil, sem problemas!

In [ ]:
# códigos...

## Passo 5 - Demais experimentos

Agora você irá testar novas configurações de campos e configurações para tentar encontrar qual a que te dá melhores resultados. Mostrar, como um relatório, a evolução do trabalho. Ou seja, mostrar como o trabalho foi evoluindo para você alcançar o melhor resultado (quais modificações foram feitas, como cada modificação influenciou nas métricas, etc).

In [ ]:
# códigos...

## Passo 6 - Quantos resultados eu devo voltar para o usuário?

Um dos problemas de sistemas de IR é determinar quantos resultados você deve retornar ao usuário. Uma forma de entender o comportamento do sistema e qual o corte ideal na lista resultante é através do uso de curvas ROC. Assim, plote a curva ROC do teu sistema e determine, através da análise da curva, qual o ponto ideal de corte para o teu sistema.

In [ ]:
# códigos...